In [1]:
from pyspark.sql.functions import input_file_name, regexp_extract, regexp_replace, col, lit, current_timestamp
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from datetime import datetime, timedelta
import configparser
import boto3
import sys
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1672984118101_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
RawSchema = StructType([
        StructField("Time-In-Secs", DoubleType(), nullable=True),
        StructField("Time-Per-Sec", StringType(), nullable=True),
        StructField("BARO-CORRECTED-PALT-DERIVED",
                    StringType(), nullable=True),
        StructField("Altitude-BaroL", StringType(), nullable=True),
        StructField("HEADING-MAGNETIC", StringType(), nullable=True),
        StructField("PITCH-ANGLE", StringType(), nullable=True),
        StructField("ROLL-ANGLE", StringType(), nullable=True),
        StructField("RAD-ALT", StringType(), nullable=True),
        StructField("WOW-NLG", StringType(), nullable=True),
        StructField("WOW-MLG", StringType(), nullable=True),
        StructField("ACCN-NORMAL", StringType(), nullable=True),
        StructField("ACCN-LATERAL", StringType(), nullable=True),
        StructField("ACCN-LONGITUDINAL", StringType(), nullable=True),
        StructField("AIRSPEED-CAS-L", StringType(), nullable=True),
        StructField("AFCS-SEL-AIRSPEED", StringType(), nullable=True),
        StructField("VERTICAL-SPEED-Derived", StringType(), nullable=True),
        StructField("FMS-GROUNDSPEED", StringType(), nullable=True),
        StructField("FLAP-CONTROL-POS-Gated", StringType(), nullable=True),
        StructField("FLAP-POSITION", StringType(), nullable=True),
        StructField("PLA-1", StringType(), nullable=True),
        StructField("PLA-2", StringType(), nullable=True),
        StructField("TORQUE-1", StringType(), nullable=True),
        StructField("TORQUE-2", StringType(), nullable=True),
        StructField("CLA-1", StringType(), nullable=True),
        StructField("CLA-2", StringType(), nullable=True),
        StructField("NP-1", StringType(), nullable=True),
        StructField("NP-2", StringType(), nullable=True),
        StructField("EGPWS-ALERT-WARNING", StringType(), nullable=True),
        StructField("EGPWS-GS-ALERT", StringType(), nullable=True),
        StructField("AFCS-GS-MODE", StringType(), nullable=True),
        StructField("AFCS-LOC-MODE", StringType(), nullable=True),
        StructField("AFCS-VOR-MODE", StringType(), nullable=True),
        StructField("ILS-GS-DEV-1", StringType(), nullable=True),
        StructField("ILS-GS-DEV-2", StringType(), nullable=True),
        StructField("FUEL-FLOW-2-kg-h", StringType(), nullable=True),
        StructField("FUEL-FLOW-1-kg-h", StringType(), nullable=True),
        StructField("DME-DISTANCE-1", StringType(), nullable=True),
        StructField("DME-DISTANCE-2", StringType(), nullable=True),
        StructField("VOR-FREQ-1", StringType(), nullable=True),
        StructField("VOR-FREQ-2", StringType(), nullable=True),
        StructField("FMS-WIND-ANGLE", StringType(), nullable=True),
        StructField("FMS-WIND-SPEED", StringType(), nullable=True),
        StructField("BRAKE-PRESS-LEFT", StringType(), nullable=True),
        StructField("BRAKE-PRESS-RIGHT", StringType(), nullable=True),
        StructField("PROP-BETA-RANGE-1", StringType(), nullable=True),
        StructField("PROP-BETA-RANGE-2", StringType(), nullable=True),
        StructField("AP-ENGAGED", StringType(), nullable=True),
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
raw_df = spark.read.csv(
        's3://sjet-datamart-bucket/PrecogFiles/Q400/512 WPS/20230104/', header=True, schema=RawSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
raw_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5419936

In [17]:
raw_df_withfilename_1 = raw_df.withColumn("FileName", regexp_extract(
                    regexp_replace(input_file_name(), '%20', ' '), "ALL\s(.*?)\.(\w+)$", 0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
raw_df_withfilename_1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5419936

In [19]:
flight_data_info = spark.read \
                            .format("csv") \
                            .option("header", "true") \
                            .load('s3://sjet-datamart-bucket/precog/precog-file-info/q400/512-wps/20230104/FileInfo_20230104.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
raw_df_withfilename_1.createOrReplaceTempView("RawData_1")
flight_data_info.dropDuplicates().createOrReplaceTempView("FlightInfo")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
raw_df_withfilename_2 = spark.sql('''

                            SELECT *,
                            
                            CASE 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 1 THEN CONCAT('00000',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 2 THEN CONCAT('0000',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 3 THEN CONCAT('000',CAST(`Time-Per-Sec` as String))
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 4 THEN CONCAT('00',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 5 THEN CONCAT('0',CAST(`Time-Per-Sec` as String))
                                ELSE `Time-Per-Sec`
                            END As Time
                                                                
                            FROM RawData_1
                                                
                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
raw_df_withfilename_3 = raw_df_withfilename_2.withColumn("Time-Per-Sec-1", regexp_replace(col("Time") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
raw_df_withfilename_3.createOrReplaceTempView('RawData_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
raw_df_withfilename_3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5419936

In [25]:
flight_data_info_With_J = spark.sql('''

                                        SELECT * FROM FlightInfo
                                        
                                        Where Filename Like ('%SEJ%J%')
                                                ''')

flight_data_info_With_J.createOrReplaceTempView('flight_data_info_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
flight_data_info_With_J.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13

In [27]:
flight_data_info_Without_J = spark.sql('''
                                            
                                            SELECT * FROM FlightInfo
                                            
                                            WHERE Filename NOT LIKE ('%SEJ%J%')

                                                    ''')

flight_data_info_Without_J.createOrReplaceTempView('flight_data_info_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
flight_data_info_Without_J.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

106

In [29]:
flight_data_join_df_Without_J = spark.sql(f'''

                            
                            SELECT
                            
                                fi.TailNo,

                                fi.Departure,

                                fi.Arrival,

                                fi.FlightNo,

                                fi.FlightDate as `Date`,

                                rw.*,

                                current_timestamp() as `LoadedDate`,

                                'precog-spark' as `LoadedBy`,

                                '512 WPS' as `DataFrameName`,

                                fi.year,

                                fi.month

                            FROM RawData_2 rw

                            JOIN flight_data_info_Without_J fi 
                            
                            ON fi.FileName = rw.FileName
            
                                        ''')

                
flight_data_join_df_Without_J.createOrReplaceTempView('flight_data_join_df_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
spark.sql('''

SELECT count(distinct(filename)) FROM flight_data_join_df_Without_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                     106|
+------------------------+

In [32]:
flight_data_join_df_With_J = spark.sql(f'''

                            
                            SELECT
                            
                            fi.TailNo,
                            
                            fi.Departure,
                            
                            fi.Arrival,
                            
                            fi.FlightNo,
                            
                            fi.FlightDate as `Date`,
                            
                            rw.*,
                            
                            current_timestamp() as `LoadedDate`,
                            
                            'precog-spark' as `LoadedBy`,
                            
                            '512 WPS' as `DataFrameName`,
                            
                            fi.year,
                            
                            fi.month

                            FROM RawData_2 rw

                            JOIN flight_data_info_With_J fi 
                            
                            ON fi.FileName = rw.FileName
                            
                            
            
                                        ''')

flight_data_join_df_With_J.createOrReplaceTempView('flight_data_join_df_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
spark.sql('''

SELECT count(distinct(filename)) FROM flight_data_join_df_With_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                      13|
+------------------------+

In [34]:
UTC_Without_J = spark.sql('''

                    SELECT TailNo, Departure, Arrival, FlightNo, Date, Filename, to_timestamp(concat(Date,' ',first_value(`Time-Per-Sec-1`,true))) AS UTC_TIMEDATE
                    
                    FROM flight_data_join_df_Without_J
                    
                    Group By TailNo, Departure, Arrival, FlightNo, Date, Filename

                            ''')

UTC_Without_J.createOrReplaceTempView('UTC_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
UTC_Without_J.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

106

In [36]:
UTC_With_J = spark.sql('''

                    SELECT TailNo, Departure, Arrival, FlightNo, Date, Filename, to_timestamp(concat(Date,' ',first_value(`Time-Per-Sec-1`,true))) AS UTC_TIMEDATE
                    
                    FROM flight_data_join_df_With_J
                    
                    Group By TailNo, Departure, Arrival, FlightNo, Date, Filename

                            ''')

UTC_With_J.createOrReplaceTempView('UTC_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
UTC_With_J.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13

In [38]:
UTC_TIMEDATE_Without_J = spark.sql('''

                SELECT
                            
                            
                            
                            rw.*,
                            
                            fi.UTC_TIMEDATE
                            
                            FROM flight_data_join_df_Without_J rw

                            JOIN UTC_Without_J fi 
                            
                            ON fi.FileName = rw.FileName



                            ''')

UTC_TIMEDATE_Without_J.createOrReplaceTempView('UTC_TIMEDATE_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
spark.sql('''

SELECT count(distinct(filename)) from UTC_TIMEDATE_Without_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                     106|
+------------------------+

In [41]:
UTC_TIMEDATE_With_J = spark.sql('''

                SELECT
                            
                            
                            
                            rw.*,
                            
                            fi.UTC_TIMEDATE
                            
                            FROM flight_data_join_df_With_J rw

                            JOIN UTC_With_J fi 
                            
                            ON fi.FileName = rw.FileName



                            ''')

UTC_TIMEDATE_With_J.createOrReplaceTempView('UTC_TIMEDATE_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
spark.sql('''

SELECT count(distinct(filename)) from UTC_TIMEDATE_With_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                      13|
+------------------------+

In [43]:
flight_data_join_df = spark.sql('''
                
                SELECT * FROM flight_data_join_df_Without_J

                UNION

                SELECT * FROM flight_data_join_df_With_J
                
                
                ''')

flight_data_join_df.createOrReplaceTempView('flight_data_join_df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
spark.sql('''

SELECT count(distinct(filename)) from flight_data_join_df

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                     119|
+------------------------+

In [45]:
MainQueryDF1 = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header = True)

MainQueryDF1.dropDuplicates().createOrReplaceTempView('MainQueryDF1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
MainQueryDF = spark.sql('''

                            SELECT 
                                    
                                    FlightID,
                                    
                                    DesigFlightNumber,
                                    
                                    AcRegNo
                                    
                                    ,FlightNumber
                                    
                                    ,FlightDate
                                    
                                    ,UTCFlightDate
                                    
                                    ,FromCity
                                    
                                    ,ToCity
                                    
                                    ,BlockTime
                                    
                                    ,ECN
                                    
                                    ,EmployeeName
                                    
                                    ,CAST (ChocksOff AS Timestamp)
                                    
                                    ,CAST (ChocksOn AS Timestamp)
                                    
                                    ,RANK
                                    
                                    ,DOMINT
                                    
                                    ,Payload
                                    
                                    ,AircraftType
                                    
                                    ,AircraftModel

                            FROM MainQueryDF1

                            ''')

MainQueryDF.dropDuplicates().createOrReplaceTempView('MainQueryDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
Arms_Join_DF_Without_J = spark.sql('''
                                            
                                            SELECT
                                                    
                                                    AD.FlightID,
                                                    
                                                    AD.DesigFlightNumber,
                                                    
                                                    AD.FlightNumber AS `FlightNumberARMS`,
                                                    
                                                    AD.FlightDate,
                                                    
                                                    AD.FromCity,
                                                    
                                                    AD.ToCity,
                                                    
                                                    AD.BlockTime,
                                                    
                                                    AD.ECN,
                                                    
                                                    AD.EmployeeName,
                                                    
                                                    AD.RANK,
                                                    
                                                    CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                                    
                                                    CAST(AD.ChocksOn AS timestamp) AS ChocksOn,
                                                    
                                                    AD.DOMINT,
                                                    
                                                    AD.AircraftType,
                                                    
                                                    AD.AircraftModel,  
                                                    
                                                    AD.Payload,
                                                    
                                                    RDF.*
                                                    
                                                    FROM UTC_TIMEDATE_Without_J RDF
                                                    
                                                    LEFT JOIN MainQueryDF AD ON RDF.FlightNo=AD.DesigFlightNumber
                                                    
                                                    AND RDF.TailNo = AD.AcRegNo
                                                    
                                                    AND RDF.Date=AD.UTCFlightDate
                                                    
                                                    AND RDF.Departure=AD.FromCity
                                                    
                                                    AND RDF.Arrival=AD.Tocity
                                                    
                                                    ''')

Arms_Join_DF_Without_J.createOrReplaceTempView('Arms_Join_DF_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
spark.sql('''

SELECT count(distinct(filename)) from Arms_Join_DF_Without_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                     106|
+------------------------+

In [49]:
Arms_Join_DF_With_J_1 = spark.sql('''

                                           SELECT
                                                    
                                                    AD.FlightID,
                                                    
                                                    AD.DesigFlightNumber,
                                                    
                                                    AD.FlightNumber AS `FlightNumberARMS`,
                                                    
                                                    AD.FlightDate,
                                                    
                                                    AD.FromCity,
                                                    
                                                    AD.ToCity,
                                                    
                                                    AD.BlockTime,
                                                    
                                                    AD.ECN,
                                                    
                                                    AD.EmployeeName,
                                                    
                                                    AD.RANK,
                                                    
                                                    CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                                    
                                                    CAST(AD.ChocksOn AS timestamp) AS ChocksOn,
                                                    
                                                    AD.DOMINT,
                                                    
                                                    AD.AircraftType,
                                                    
                                                    AD.AircraftModel,  
                                                    
                                                    AD.Payload,
                                                    
                                                    RDF.*
                                                    
                                                    FROM UTC_TIMEDATE_With_J RDF
                                                    
                                                    LEFT JOIN MainQueryDF AD ON 
                                                    
                                                    RDF.TailNo = AD.AcRegNo
                                                    
                                                    AND RDF.Date = AD.UTCFlightDate
                                                    
                                                    AND RDF.UTC_TIMEDATE BETWEEN (AD.ChocksOff - INTERVAL 345 MINUTES) AND (AD.ChocksOff - INTERVAL 315 MINUTES)
                                                    
                                                    AND RDF.Departure=AD.FromCity
                                                    
                                                    AND RDF.Arrival=AD.Tocity
                                                                        
                                                ''')

Arms_Join_DF_With_J_1.createOrReplaceTempView('Arms_Join_DF_With_J_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
spark.sql('''

SELECT count(distinct(filename)) from Arms_Join_DF_With_J_1

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                      13|
+------------------------+

In [51]:
Arms_Join_DF_With_J = spark.sql('''
                                                SELECT 
                                                    FlightID, 
                                                    DesigFlightNumber, 
                                                    FlightNumberARMS, 
                                                    FlightDate, 
                                                    FromCity, 
                                                    ToCity, 
                                                    BlockTime, 
                                                    ECN, 
                                                    EmployeeName, 
                                                    RANK, 
                                                    ChocksOff, 
                                                    ChocksOn, 
                                                    DOMINT, 
                                                    AircraftType, 
                                                    AircraftModel, 
                                                    Payload, 
                                                    TailNo, 
                                                    Departure, 
                                                    Arrival, 
                                                    FlightNo, 
                                                    Date, 
                                                    `Time-In-Secs`,
                                                    `Time-Per-Sec`, 
                                                    `BARO-CORRECTED-PALT-DERIVED`, 
                                                    `Altitude-BaroL`, 
                                                    `HEADING-MAGNETIC`, 
                                                    `PITCH-ANGLE`, 
                                                    `ROLL-ANGLE`, 
                                                    `RAD-ALT`, 
                                                    `WOW-NLG`, 
                                                    `WOW-MLG`, 
                                                    `ACCN-NORMAL`, 
                                                    `ACCN-LATERAL`, 
                                                    `ACCN-LONGITUDINAL`, 
                                                    `AIRSPEED-CAS-L`, 
                                                    `AFCS-SEL-AIRSPEED`, 
                                                    `VERTICAL-SPEED-Derived`, 
                                                    `FMS-GROUNDSPEED`, 
                                                    `FLAP-CONTROL-POS-Gated`, 
                                                    `FLAP-POSITION`, 
                                                    `PLA-1`, 
                                                    `PLA-2`, 
                                                    `TORQUE-1`, 
                                                    `TORQUE-2`, 
                                                    `CLA-1`, 
                                                    `CLA-2`, 
                                                    `NP-1`, 
                                                    `NP-2`, 
                                                    `EGPWS-ALERT-WARNING`, 
                                                    `EGPWS-GS-ALERT`, 
                                                    `AFCS-GS-MODE`, 
                                                    `AFCS-LOC-MODE`, 
                                                    `AFCS-VOR-MODE`, 
                                                    `ILS-GS-DEV-1`, 
                                                    `ILS-GS-DEV-2`, 
                                                    `FUEL-FLOW-2-kg-h`, 
                                                    `FUEL-FLOW-1-kg-h`, 
                                                    `DME-DISTANCE-1`, 
                                                    `DME-DISTANCE-2`, 
                                                    `VOR-FREQ-1`, 
                                                    `VOR-FREQ-2`, 
                                                    `FMS-WIND-ANGLE`, 
                                                    `FMS-WIND-SPEED`, 
                                                    `BRAKE-PRESS-LEFT`, 
                                                    `BRAKE-PRESS-RIGHT`, 
                                                    `PROP-BETA-RANGE-1`, 
                                                    `PROP-BETA-RANGE-2`, 
                                                    `AP-ENGAGED`, 
                                                    FileName, 
                                                    Time, 
                                                    `Time-Per-Sec-1`, 
                                                    LoadedDate,
                                                    LoadedBy, 
                                                    DataFrameName, 
                                                    year, 
                                                    month,
                                                    UTC_TIMEDATE

                                                    FROM Arms_Join_DF_With_J_1

                                            ''')

Arms_Join_DF_With_J.createOrReplaceTempView('Arms_Join_DF_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
spark.sql('''

SELECT count(distinct(filename)) from Arms_Join_DF_With_J

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT filename)|
+------------------------+
|                      13|
+------------------------+

In [53]:
Arms_Join_DF = spark.sql('''

                                SELECT * FROM Arms_Join_DF_Without_J
                                
                                UNION
                                
                                SELECT * FROM Arms_Join_DF_With_J
                

                    ''')

Arms_Join_DF.createOrReplaceTempView('Arms_Join_DF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
spark.sql('''

SELECT distinct(LoadedDate) FROM Arms_Join_DF where filename = 'ALL ac=VT-SQB dep=BOM arr=IXY fn=SG 2903 dt=20230104.csv'


''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|          LoadedDate|
+--------------------+
|2023-01-06 09:37:...|
+--------------------+

In [57]:
flight_data_info.select('Year', 'Month').distinct().createOrReplaceTempView('flight_data_info')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
PartitionList=spark.sql(''' SELECT CONCAT(Year,Month) AS YearMonth FROM flight_data_info ''').rdd.flatMap(lambda x: x).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
Partition=str(PartitionList)[1:len(str(PartitionList))-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
ExistingDataDF = spark.sql(f'''

                                SELECT   *
                                FROM Arms_Join_DF
                                WHERE CONCAT(Year,Month) 
                                IN ({Partition})
                                                                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
ExistingDataDF.createOrReplaceTempView('ExistingDataDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
spark.sql('''

SELECT * FROM ExistingDataDF where filename = 'ALL ac=VT-SQB dep=BOM arr=IXY fn=SG 2903 dt=20230104.csv'



''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+-----------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+-----------+----------+-------+----------+----------+-----------+------------+-----------------+--------------+-----------------+----------------------+---------------+----------------------+-------------+-----+-----+--------+--------+-----+-----+----+----+-------------------+--------------+--------------+--------------+--------------+------------+------------+----------------+----------------+--------------+--------------+----------+----------+--------------+--------------+----------------+-----------------+-----------------+-----------------+--------------+--------------------+------+--------------+--------------------+------------+-------------+----+-------+-----

In [ ]:
FileLPD = spark.sql('''
                            SELECT TailNo,
                                FlightNumberARMS,
                                Date,
                                Departure,
                                Arrival,
                                LoadedDate,
                                FileName, 
                                Count,
                                ROW_NUMBER() OVER(PARTITION BY TailNo,
                                                                FlightNumberARMS,
                                                                Date,
                                                                Departure,
                                                                Arrival
                                                                ORDER BY count DESC,
                                                                LoadedDate DESC) AS RN
                            FROM(
                                SELECT 
                                        TailNo,
                                        FlightNumberARMS,
                                        Date,
                                        Departure,
                                        Arrival,
                                        LoadedDate,
                                        FileName,
                                        Count(*) AS Count

                                FROM ExistingDataDF
                                WHERE FileName NOT LIKE '%(_).csv'
                                GROUP BY TailNo,
                                            FlightNumberARMS,
                                            Date,
                                            Departure,
                                            Arrival,
                                            FileName,
                                            LoadedDate)
                        
                                                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
spark.sql('''
SELECT 
                                        TailNo,
                                        FlightNumberARMS,
                                        Date,
                                        Departure,
                                        Arrival,
                                        LoadedDate,
                                        FileName,
                                        Count(*) AS Count

                                FROM ExistingDataDF
                                WHERE FileName NOT LIKE '%(_).csv'
                                GROUP BY TailNo,
                                            FlightNumberARMS,
                                            Date,
                                            Departure,
                                            Arrival,
                                            FileName,
                                            LoadedDate 
                                            ''').where(col('Filename') == 'ALL ac=VT-SQB dep=BOM arr=IXY fn=SG 2903 dt=20230104.csv').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+----------+---------+-------+---------------------+--------------------------------------------------------+-----+
|TailNo|FlightNumberARMS|Date      |Departure|Arrival|LoadedDate           |FileName                                                |Count|
+------+----------------+----------+---------+-------+---------------------+--------------------------------------------------------+-----+
|VT-SQB|2903            |2023-01-04|BOM      |IXY    |2023-01-06 09:35:18.3|ALL ac=VT-SQB dep=BOM arr=IXY fn=SG 2903 dt=20230104.csv|37600|
+------+----------------+----------+---------+-------+---------------------+--------------------------------------------------------+-----+

In [93]:
FileLPD.createOrReplaceTempView('FileLPD')

NoDupDF = FileLPD.where(FileLPD.RN == 1)

NoDupDF.createOrReplaceTempView('NoDupDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
spark.sql('''

SELECT * FROM FileLPD where filename = 'ALL ac=VT-SQB dep=BOM arr=IXY fn=SG 2903 dt=20230104.csv'

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+----------+---------+-------+--------------------+--------------------+-----+---+
|TailNo|FlightNumberARMS|      Date|Departure|Arrival|          LoadedDate|            FileName|Count| RN|
+------+----------------+----------+---------+-------+--------------------+--------------------+-----+---+
|VT-SQB|            2903|2023-01-04|      BOM|    IXY|2023-01-06 09:26:...|ALL ac=VT-SQB dep...|37600|  2|
+------+----------------+----------+---------+-------+--------------------+--------------------+-----+---+